### 1)Import the dataset

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
data=pd.read_csv('/content/Tweets.csv')

In [ ]:
data.head()

,selected_text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD,negative
2,bullying me,negative
3,leave me alone,negative
4,"Sons of ****,",negative


In [ ]:
columan_name = ['Review', 'Sentiment']
data.columns = columan_name

In [ ]:
data.shape

(27481, 2)

In [ ]:
data["Sentiment"].value_counts()

Sentiment
neutral     11118
positive     8582
negative     7781
Name: count, dtype: int64

In [ ]:
# prompt: delete the null

data = data.dropna()


In [ ]:
data.isnull().sum()

Review       0
Sentiment    0
dtype: int64

In [ ]:
x=data['Review']
y=data['Sentiment']

In [ ]:
data.head()

,Review,Sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD,negative
2,bullying me,negative
3,leave me alone,negative
4,"Sons of ****,",negative


# 2)Data Cleaning

In [ ]:
import string

In [ ]:
punct = string.punctuation

In [ ]:
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
from spacy.lang.en.stop_words import STOP_WORDS

In [ ]:
Stopwords = list(STOP_WORDS)

In [ ]:

import spacy
nlp = spacy.load('en_core_web_sm')
def text_data_cleaning(sentence):
  doc = nlp(sentence)
  tokens = []
  for token in doc:
    if token.lemma_ != "-PRON-":
      temp = token.lemma_.lower().strip()
    else:
      temp = token.lower_
    tokens.append(temp)
  cleaned_tokens = []
  for token in tokens:
    if token not in Stopwords and token not in punct:
      cleaned_tokens.append(token)
  return cleaned_tokens

In [ ]:
text_data_cleaning("i hate you")

['hate']

# Vectorization Feature Engineering(TF-IDF)

In [ ]:
#from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [ ]:
# prompt: use naive bias instead of Linear SVC

from sklearn.naive_bayes import MultinomialNB

clf = Pipeline([('tfidf', TfidfVectorizer()),
                 ('clf', MultinomialNB())])
clf.fit(x, y)


Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [ ]:
tfidf= TfidfVectorizer(tokenizer=text_data_cleaning)

In [ ]:
classifier = MultinomialNB()

# 3)Training the model

Splitting the dataset into the Train and Test set

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [ ]:
x_train.shape,x_test.shape

((21984,), (5496,))

In [ ]:
x_train.head(5)

24567    Cant stop playin` in my head  -- pussycat doll...
24619                                           I hate you
19766                              Starbucks I`m lovin` it
21737                                            .yummmmy!
8980             wow.. purple leopard skin.  fieeerrceee..
Name: Review, dtype: object

# Fit the x_train and y_train

In [ ]:
clf = Pipeline([('tfidf',tfidf),('clf',classifier)])

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

# Assuming 'x_train' is a pandas DataFrame or Series
x_train = x_train.fillna('')  # Replace missing values with empty strings

# Continue with your existing code:
clf = Pipeline([('tfidf',tfidf),('clf',classifier)])
clf.fit(x_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function text_data_cleaning at 0x7b22330c6830>)),
                ('clf', MultinomialNB())])

In [ ]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [ ]:
y_pred = clf.predict(x_test)

In [ ]:
confusion_matrix(y_test,y_pred)

array([[1121,  286,  116],
       [ 255, 1821,  199],
       [ 135,  195, 1368]])

In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

    negative       0.74      0.74      0.74      1523
     neutral       0.79      0.80      0.80      2275
    positive       0.81      0.81      0.81      1698

    accuracy                           0.78      5496
   macro avg       0.78      0.78      0.78      5496
weighted avg       0.78      0.78      0.78      5496



In [ ]:
accuracy_score(y_test,y_pred)

0.7842066957787481

In [ ]:
clf.predict(["happy"])

array(['positive'], dtype='<U8')